In [53]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
torch.manual_seed(0) 

import pandas as pd
import random
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

import os

import numpy as np
import torch.nn.functional as F

from google.colab import drive
drive.mount('/content/drive')

from sklearn.metrics import accuracy_score
from datetime import datetime
from IPython.display import clear_output 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

1.12.1+cu113


In [55]:
train_dataset=[]
graphlist=[]
import networkx as nx
import torch_geometric
from torch_geometric.data import Data, Dataset
from torch_geometric.loader import DataLoader
#from torch.utils.data import DataLoader
from torch_geometric.utils import degree
from torch_geometric.utils import erdos_renyi_graph, to_networkx, from_networkx


%cd /content/drive/MyDrive/ADNI2/Controls
for i in range(1, 110):
    if os.path.isfile('{0:0>3}'.format(i)+'.txt'):
      tmp=pd.read_csv('{0:0>3}'.format(i)+'.txt', header=None)
      tmp=(tmp.to_numpy())
      tmp=torch.from_numpy(tmp)  
      tmp[tmp<=0.4]=0
      #print([tmp.type(torch.FloatTensor),torch.ones(1).type(torch.long)])
      G = nx.from_numpy_matrix(np.array(tmp))
      centrality = nx.clustering(G)
      x = []
      for i in range(len(centrality)):
        x.append([centrality[i]])
      x = np.array(x).reshape(90,1)
      data = from_networkx(G)
      #x = torch.tensor([[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1] ], dtype=torch.float)
      data.x = torch.FloatTensor(x)
      data.y = torch.tensor([1])
      graphlist.append(data)
      #graphlist.append([tmp.type(torch.FloatTensor),torch.ones(1).type(torch.long)])

%cd /content/drive/MyDrive/ADNI2/Patients
for i in range(1, 110):
    if os.path.isfile('{0:0>3}'.format(i)+'.txt'):
      tmp=pd.read_csv('{0:0>3}'.format(i)+'.txt', header=None)
      tmp=(tmp.to_numpy())
      tmp=torch.from_numpy(tmp) 
      tmp[tmp<=0.4]=0
      #print([tmp.type(torch.FloatTensor),torch.ones(1).type(torch.long)])
      G = nx.from_numpy_matrix(np.array(tmp))
      centrality = nx.clustering(G)
      x = []
      for i in range(len(centrality)):
        x.append([centrality[i]])
      x = np.array(x).reshape(90,1)
      data = from_networkx(G)
      #x = torch.tensor([[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1] ], dtype=torch.float)
      data.x = torch.FloatTensor(x)
      data.y = torch.tensor([0])
      graphlist.append(data)
      #graphlist.append([data, torch.zeros(1).type(torch.long)])

      #graphlist.append([tmp.type(torch.FloatTensor),torch.zeros(1).type(torch.long)])

/content/drive/.shortcut-targets-by-id/1n9P-S3rsi_g9yeb5KW1h7v-Bte7Cy43G/ADNI2/Controls
/content/drive/.shortcut-targets-by-id/1n9P-S3rsi_g9yeb5KW1h7v-Bte7Cy43G/ADNI2/Patients


In [56]:
dataset = random.sample(graphlist, len(graphlist))
train_dataset = dataset[:150]
test_dataset = dataset[150:]
train_loader = DataLoader(train_dataset, batch_size=25, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=25, shuffle=False)

In [57]:
from torch_geometric.nn import GATConv, GraphConv
from torch_geometric.nn.conv.gatv2_conv import GATv2Conv
from torch_geometric.nn import global_mean_pool, global_sort_pool

class GNN(torch.nn.Module):
    def __init__(self):
        super(GNN, self).__init__()

        self.in_head = 8
        self.out_head = 1
        self.conv1 = GraphConv(in_channels=1, out_channels=16)
        self.conv2 = GraphConv(in_channels=16, out_channels=32)
        self.conv3 = GraphConv(in_channels=32, out_channels=32)
        self.lin1 = torch.nn.Linear(32, 16)
        self.lin2 = torch.nn.Linear(16, 1)
        self.sig = torch.nn.Sigmoid()

    def forward(self, data):
        x, edge_index, weight, batch = data.x, data.edge_index, data.weight, data.batch 
        x = self.conv1(x, edge_index, weight)
        x = F.relu(x)
        #print(x.size())
        x = self.conv2(x, edge_index, weight)
        x = F.relu(x)
        #print(x.size())
        x = self.conv3(x, edge_index, weight)
        x = F.relu(x)
        #print(x.size())
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        #x = F.dropout(x, p=0.2, training=self.training)
        x = self.lin1(x)
        x = self.lin2(x)
        #print(x)
        
        return x
        #return F.log_softmax(x, dim=1)

In [58]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNN().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4600, Test Acc: 0.5882, Loss: 238.8990
Epoch: 002, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.6414
Epoch: 003, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.6943
Epoch: 004, Train Acc: 0.5267, Test Acc: 0.6618, Loss: 0.6728
Epoch: 005, Train Acc: 0.5533, Test Acc: 0.4412, Loss: 0.6395
Epoch: 006, Train Acc: 0.7067, Test Acc: 0.6176, Loss: 0.8739
Epoch: 007, Train Acc: 0.8133, Test Acc: 0.8382, Loss: 0.4267
Epoch: 008, Train Acc: 0.9133, Test Acc: 0.9265, Loss: 0.6410
Epoch: 009, Train Acc: 0.8533, Test Acc: 0.8676, Loss: 0.3576
Epoch: 010, Train Acc: 0.9400, Test Acc: 0.9412, Loss: 0.5189
Epoch: 011, Train Acc: 0.9000, Test Acc: 0.8676, Loss: 0.1428
Epoch: 012, Train Acc: 0.9000, Test Acc: 0.9265, Loss: 0.4163
Epoch: 013, Train Acc: 0.9000, Test Acc: 0.8676, Loss: 0.1141
Epoch: 014, Train Acc: 0.9133, Test Acc: 0.9118, Loss: 0.2090
Epoch: 015, Train Acc: 0.9200, Test Acc: 0.8676, Loss: 0.1785
Epoch: 016, Train Acc: 0.9333, Test Acc: 0.8824, Loss: 0.0923
Epoch:

In [59]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import GCNConv, JumpingKnowledge, global_mean_pool


class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.num_layers  = 3
        self.hidden = 16
        self.conv1 = GCNConv(1, self.hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.convs.append(GCNConv(self.hidden, self.hidden))
        self.lin1 = Linear(self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index,weight, batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index, weight))
        x = global_mean_pool(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

In [60]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4600, Test Acc: 0.5882, Loss: 0.6851
Epoch: 002, Train Acc: 0.6533, Test Acc: 0.6912, Loss: 0.6945
Epoch: 003, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.6860
Epoch: 004, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.7009
Epoch: 005, Train Acc: 0.5000, Test Acc: 0.5735, Loss: 0.6809
Epoch: 006, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.6694
Epoch: 007, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.6702
Epoch: 008, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.6971
Epoch: 009, Train Acc: 0.8333, Test Acc: 0.7794, Loss: 0.6349
Epoch: 010, Train Acc: 0.6667, Test Acc: 0.6618, Loss: 0.6548
Epoch: 011, Train Acc: 0.7533, Test Acc: 0.7647, Loss: 0.6381
Epoch: 012, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.6520
Epoch: 013, Train Acc: 0.5733, Test Acc: 0.5882, Loss: 0.5784
Epoch: 014, Train Acc: 0.5400, Test Acc: 0.4265, Loss: 0.7151
Epoch: 015, Train Acc: 0.6933, Test Acc: 0.7500, Loss: 0.6641
Epoch: 016, Train Acc: 0.5600, Test Acc: 0.4853, Loss: 0.7160
Epoch: 0

In [61]:
from torch.nn import Linear
from torch_geometric.nn import GINConv, JumpingKnowledge, global_mean_pool
from torch.nn import BatchNorm1d as BN
from torch.nn import Linear, ReLU, Sequential
#from torch_geometric.nn import JumpingKnowledge, SAGEConv, global_mean_pool

class GIN(torch.nn.Module):
    def __init__(self):
        super(GIN, self).__init__()
        self.conv1 = GINConv(
            Sequential(
                Linear(1, 16),
                ReLU(),
                Linear(16, 16),
                ReLU(),
                BN(16),
            ), train_eps=False)
        self.conv2 = GINConv(
            Sequential(
                Linear(16, 32),
                ReLU(),
                Linear(32, 32),
                ReLU(),
                BN(32),
            ), train_eps=False)
        self.conv3 = GINConv(
            Sequential(
                Linear(32, 32),
                ReLU(),
                Linear(32, 32),
                ReLU(),
                BN(32),
            ), train_eps=False)
        self.lin1 = Linear(32, 16)
        self.lin2 = Linear(16, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        x = self.conv3(x, edge_index)
        x = global_mean_pool(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)

In [62]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GIN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4667, Test Acc: 0.5882, Loss: 0.5247
Epoch: 002, Train Acc: 0.5333, Test Acc: 0.5735, Loss: 0.3095
Epoch: 003, Train Acc: 0.6800, Test Acc: 0.7206, Loss: 0.8103
Epoch: 004, Train Acc: 0.7600, Test Acc: 0.7647, Loss: 0.4241
Epoch: 005, Train Acc: 0.7600, Test Acc: 0.7794, Loss: 0.4474
Epoch: 006, Train Acc: 0.7800, Test Acc: 0.7794, Loss: 0.4480
Epoch: 007, Train Acc: 0.8467, Test Acc: 0.8088, Loss: 0.2363
Epoch: 008, Train Acc: 0.8267, Test Acc: 0.7941, Loss: 0.4749
Epoch: 009, Train Acc: 0.8333, Test Acc: 0.8235, Loss: 0.3010
Epoch: 010, Train Acc: 0.8400, Test Acc: 0.7500, Loss: 0.4008
Epoch: 011, Train Acc: 0.8400, Test Acc: 0.7500, Loss: 0.5171
Epoch: 012, Train Acc: 0.8600, Test Acc: 0.8382, Loss: 0.5754
Epoch: 013, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.3257
Epoch: 014, Train Acc: 0.6867, Test Acc: 0.7206, Loss: 0.5122
Epoch: 015, Train Acc: 0.7533, Test Acc: 0.7647, Loss: 0.2909
Epoch: 016, Train Acc: 0.7600, Test Acc: 0.7794, Loss: 0.3860
Epoch: 0

In [63]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.data import Batch
from torch_geometric.nn import (
    GraphConv,
    JumpingKnowledge,
    global_mean_pool,
    graclus,
    max_pool,
)

num_layers  = 4
hidden = 16
class Graclus(torch.nn.Module):
    def __init__(self):
        super(Graclus, self).__init__()
        self.conv1 = GraphConv(1, 16, aggr='mean')
        self.convs = torch.nn.ModuleList()
        for i in range(num_layers - 1):
            self.convs.append(GraphConv(16, 16, aggr='mean'))
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(64, 16)
        self.lin2 = Linear(16, 1)

    def forward(self, data):
        x, edge_index, weight , batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                cluster = graclus(edge_index,num_nodes=x.size(0))
                data = Batch(x=x, edge_index=edge_index, batch=batch)
                data = max_pool(cluster, data)
                x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        x = self.lin2(x)
        return x

In [64]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Graclus().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.7042
Epoch: 002, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.6659
Epoch: 003, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.7018
Epoch: 004, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.7150
Epoch: 005, Train Acc: 0.5467, Test Acc: 0.4412, Loss: 0.6539
Epoch: 006, Train Acc: 0.5733, Test Acc: 0.5294, Loss: 0.6748
Epoch: 007, Train Acc: 0.6400, Test Acc: 0.5441, Loss: 0.5689
Epoch: 008, Train Acc: 0.8867, Test Acc: 0.8529, Loss: 0.4890
Epoch: 009, Train Acc: 0.8667, Test Acc: 0.8529, Loss: 0.4828
Epoch: 010, Train Acc: 0.8800, Test Acc: 0.8382, Loss: 0.5514
Epoch: 011, Train Acc: 0.8933, Test Acc: 0.8676, Loss: 0.4162
Epoch: 012, Train Acc: 0.8933, Test Acc: 0.8676, Loss: 0.2572
Epoch: 013, Train Acc: 0.8867, Test Acc: 0.8676, Loss: 0.4082
Epoch: 014, Train Acc: 0.9000, Test Acc: 0.8971, Loss: 0.2328
Epoch: 015, Train Acc: 0.8800, Test Acc: 0.8824, Loss: 0.2902
Epoch: 016, Train Acc: 0.9133, Test Acc: 0.9118, Loss: 0.2120
Epoch: 0

In [65]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    GraphConv,
    JumpingKnowledge,
    TopKPooling,
    global_mean_pool,
)


class TopK(torch.nn.Module):
    def __init__(self):
        super(TopK, self).__init__()
        self.ratio = 0.8
        self.num_layers = 3
        self.hidden = 16
        self.conv1 = GraphConv(1, 16, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(16, 16, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend(
            [TopKPooling(16, self.ratio) for i in range((self.num_layers) // 2)])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(48, 16)
        self.lin2 = Linear(16, 1)


    def forward(self, data):
        x, edge_index, weight, batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, _, batch, _, _ = pool(x, edge_index,
                                                     batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        #return F.log_softmax(x, dim=-1)
        return x

In [66]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TopK().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.6955
Epoch: 002, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.6989
Epoch: 003, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.6714
Epoch: 004, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.6817
Epoch: 005, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.6726
Epoch: 006, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.5734
Epoch: 007, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.6013
Epoch: 008, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.7046
Epoch: 009, Train Acc: 0.8133, Test Acc: 0.7353, Loss: 0.6275
Epoch: 010, Train Acc: 0.7933, Test Acc: 0.7206, Loss: 0.4554
Epoch: 011, Train Acc: 0.7800, Test Acc: 0.7647, Loss: 0.5676
Epoch: 012, Train Acc: 0.7067, Test Acc: 0.7647, Loss: 0.6107
Epoch: 013, Train Acc: 0.6067, Test Acc: 0.5588, Loss: 0.5063
Epoch: 014, Train Acc: 0.8067, Test Acc: 0.7794, Loss: 0.5479
Epoch: 015, Train Acc: 0.7867, Test Acc: 0.7353, Loss: 0.5214
Epoch: 016, Train Acc: 0.8267, Test Acc: 0.7794, Loss: 0.6022
Epoch: 0

In [67]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    GraphConv,
    JumpingKnowledge,
    SAGPooling,
    global_mean_pool,
)


class SAGPool(torch.nn.Module):
    def __init__(self):
        super(SAGPool, self).__init__()
        self.num_layers = 4
        self.hidden = 32
        self.ratio = 0.8
        self.conv1 = GraphConv(1, self.hidden, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(self.hidden, self.hidden, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend(
            [SAGPooling(self.hidden, self.ratio) for i in range((self.num_layers) // 2)])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(self.num_layers * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, weight, batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, _, batch, _, _ = pool(x, edge_index,
                                                     batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)

In [68]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SAGPool().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.6773
Epoch: 002, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.6863
Epoch: 003, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.6884
Epoch: 004, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.6190
Epoch: 005, Train Acc: 0.6733, Test Acc: 0.6324, Loss: 0.5547
Epoch: 006, Train Acc: 0.8733, Test Acc: 0.8529, Loss: 0.5867
Epoch: 007, Train Acc: 0.8800, Test Acc: 0.8529, Loss: 0.4146
Epoch: 008, Train Acc: 0.9000, Test Acc: 0.9118, Loss: 0.6090
Epoch: 009, Train Acc: 0.8733, Test Acc: 0.8824, Loss: 0.5167
Epoch: 010, Train Acc: 0.9000, Test Acc: 0.9412, Loss: 0.3392
Epoch: 011, Train Acc: 0.8867, Test Acc: 0.9118, Loss: 0.3764
Epoch: 012, Train Acc: 0.8667, Test Acc: 0.8824, Loss: 0.3280
Epoch: 013, Train Acc: 0.9067, Test Acc: 0.8971, Loss: 0.9595
Epoch: 014, Train Acc: 0.9067, Test Acc: 0.8824, Loss: 0.4294
Epoch: 015, Train Acc: 0.9067, Test Acc: 0.9706, Loss: 0.6038
Epoch: 016, Train Acc: 0.8667, Test Acc: 0.8824, Loss: 0.2713
Epoch: 0

In [69]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    EdgePooling,
    GraphConv,
    JumpingKnowledge,
    global_mean_pool,
)


class EdgePool(torch.nn.Module):
    def __init__(self):
        super(EdgePool, self).__init__()
        self.num_layers = 3 
        self.hidden = 16
        self.conv1 = GraphConv(1, self.hidden, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(self.hidden, self.hidden, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend(
            [EdgePooling(self.hidden) for i in range((self.num_layers) // 2)])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(self.num_layers * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, weight ,  batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, batch, _ = pool(x, edge_index, batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

In [70]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EdgePool().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.6796
Epoch: 002, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.6895
Epoch: 003, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.7007
Epoch: 004, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.6347
Epoch: 005, Train Acc: 0.5400, Test Acc: 0.4265, Loss: 0.6449
Epoch: 006, Train Acc: 0.8667, Test Acc: 0.7941, Loss: 0.5768
Epoch: 007, Train Acc: 0.8333, Test Acc: 0.8382, Loss: 0.4649
Epoch: 008, Train Acc: 0.8400, Test Acc: 0.8676, Loss: 0.4784
Epoch: 009, Train Acc: 0.8867, Test Acc: 0.8088, Loss: 0.4679
Epoch: 010, Train Acc: 0.8600, Test Acc: 0.8971, Loss: 0.2526
Epoch: 011, Train Acc: 0.9000, Test Acc: 0.8824, Loss: 0.4587
Epoch: 012, Train Acc: 0.8867, Test Acc: 0.8971, Loss: 0.2900
Epoch: 013, Train Acc: 0.9200, Test Acc: 0.9265, Loss: 0.2441
Epoch: 014, Train Acc: 0.8933, Test Acc: 0.8971, Loss: 0.3079
Epoch: 015, Train Acc: 0.9000, Test Acc: 0.8971, Loss: 0.2696
Epoch: 016, Train Acc: 0.9200, Test Acc: 0.9559, Loss: 0.5765
Epoch: 0

In [71]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import GlobalAttention, SAGEConv


class GlobalAttentionNet(torch.nn.Module):
    def __init__(self):
        super(GlobalAttentionNet, self).__init__()
        self.num_layers = 3
        self.hidden  = 16
        self.conv1 = SAGEConv(1, self.hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.convs.append(SAGEConv(self.hidden, self.hidden))
        self.att = GlobalAttention(Linear(self.hidden, 1))
        self.lin1 = Linear(self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
        x = self.att(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)


In [72]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GlobalAttentionNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.7019


/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'nn.glob.GlobalAttention' is deprecated, use 'nn.aggr.AttentionalAggregation' instead
  warnings.warn(out)


Epoch: 002, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.6798
Epoch: 003, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.6907
Epoch: 004, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.6945
Epoch: 005, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.6968
Epoch: 006, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.6545
Epoch: 007, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.6999
Epoch: 008, Train Acc: 0.6200, Test Acc: 0.5000, Loss: 0.5751
Epoch: 009, Train Acc: 0.6200, Test Acc: 0.5000, Loss: 0.5706
Epoch: 010, Train Acc: 0.8467, Test Acc: 0.8676, Loss: 0.3944
Epoch: 011, Train Acc: 0.7867, Test Acc: 0.8529, Loss: 0.3017
Epoch: 012, Train Acc: 0.8600, Test Acc: 0.8676, Loss: 0.4737
Epoch: 013, Train Acc: 0.8333, Test Acc: 0.8529, Loss: 0.5080
Epoch: 014, Train Acc: 0.8733, Test Acc: 0.8529, Loss: 0.3768
Epoch: 015, Train Acc: 0.8333, Test Acc: 0.8529, Loss: 0.3313
Epoch: 016, Train Acc: 0.8867, Test Acc: 0.8676, Loss: 0.3160
Epoch: 017, Train Acc: 0.8600, Test Acc: 0.8676, Loss: 0.3164
Epoch: 0

In [73]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import SAGEConv, Set2Set


class Set2SetNet(torch.nn.Module):
    def __init__(self):
        super(Set2SetNet,self).__init__()
        self.hidden = 16
        self.num_layers = 3 
        self.conv1 = SAGEConv(1, self.hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.convs.append(SAGEConv(self.hidden, self.hidden))
        self.set2set = Set2Set(self.hidden, processing_steps=4)
        self.lin1 = Linear(2 * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
        x = self.set2set(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

In [74]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Set2SetNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'nn.glob.Set2Set' is deprecated, use 'nn.aggr.Set2Set' instead
  warnings.warn(out)


Epoch: 001, Train Acc: 0.4600, Test Acc: 0.5882, Loss: 0.7032
Epoch: 002, Train Acc: 0.4600, Test Acc: 0.5882, Loss: 0.7025
Epoch: 003, Train Acc: 0.4600, Test Acc: 0.5882, Loss: 0.7085
Epoch: 004, Train Acc: 0.4600, Test Acc: 0.5882, Loss: 0.6894
Epoch: 005, Train Acc: 0.4600, Test Acc: 0.5882, Loss: 0.6925
Epoch: 006, Train Acc: 0.4600, Test Acc: 0.5882, Loss: 0.7005
Epoch: 007, Train Acc: 0.4600, Test Acc: 0.5882, Loss: 0.6975
Epoch: 008, Train Acc: 0.4600, Test Acc: 0.5882, Loss: 0.6901
Epoch: 009, Train Acc: 0.4600, Test Acc: 0.5882, Loss: 0.6969
Epoch: 010, Train Acc: 0.4600, Test Acc: 0.5882, Loss: 0.6865
Epoch: 011, Train Acc: 0.4600, Test Acc: 0.5882, Loss: 0.6964
Epoch: 012, Train Acc: 0.4600, Test Acc: 0.5882, Loss: 0.6943
Epoch: 013, Train Acc: 0.4600, Test Acc: 0.5882, Loss: 0.6942
Epoch: 014, Train Acc: 0.4600, Test Acc: 0.5882, Loss: 0.6942
Epoch: 015, Train Acc: 0.4600, Test Acc: 0.5882, Loss: 0.6954
Epoch: 016, Train Acc: 0.4600, Test Acc: 0.5882, Loss: 0.6926
Epoch: 0

In [75]:
import torch
import torch.nn.functional as F
from torch.nn import Conv1d, Linear

from torch_geometric.nn import SAGEConv, global_sort_pool


class SortPool(torch.nn.Module):
    def __init__(self):
        super(SortPool, self).__init__()
        self.k = 30
        self.num_layers = 3 
        self.hidden = 16
        self.conv1 = SAGEConv(1, self.hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.convs.append(SAGEConv(self.hidden, self.hidden))
        self.conv1d = Conv1d(self.hidden, 32, 5)
        self.lin1 = Linear(32 * (self.k - 5 + 1), self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
        x = global_sort_pool(x, batch, self.k)
        x = x.view(len(x), self.k, -1).permute(0, 2, 1)
        x = F.relu(self.conv1d(x))
        x = x.view(len(x), -1)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

In [76]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SortPool().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 200):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'nn.glob.global_sort_pool' is deprecated, use 'nn.aggr.SortAggr' instead
  warnings.warn(out)


Epoch: 001, Train Acc: 0.4600, Test Acc: 0.5882, Loss: 0.7226
Epoch: 002, Train Acc: 0.4600, Test Acc: 0.5882, Loss: 0.6860
Epoch: 003, Train Acc: 0.4600, Test Acc: 0.5882, Loss: 0.6794
Epoch: 004, Train Acc: 0.4600, Test Acc: 0.5882, Loss: 0.7170
Epoch: 005, Train Acc: 0.4600, Test Acc: 0.5882, Loss: 0.6688
Epoch: 006, Train Acc: 0.4600, Test Acc: 0.5882, Loss: 0.6973
Epoch: 007, Train Acc: 0.4600, Test Acc: 0.5882, Loss: 0.6917
Epoch: 008, Train Acc: 0.4600, Test Acc: 0.5882, Loss: 0.7064
Epoch: 009, Train Acc: 0.4600, Test Acc: 0.5882, Loss: 0.7006
Epoch: 010, Train Acc: 0.4600, Test Acc: 0.5882, Loss: 0.7032
Epoch: 011, Train Acc: 0.4600, Test Acc: 0.5882, Loss: 0.7011
Epoch: 012, Train Acc: 0.4600, Test Acc: 0.5882, Loss: 0.6878
Epoch: 013, Train Acc: 0.4600, Test Acc: 0.5882, Loss: 0.6864
Epoch: 014, Train Acc: 0.4600, Test Acc: 0.5882, Loss: 0.6922
Epoch: 015, Train Acc: 0.4933, Test Acc: 0.5735, Loss: 0.6941
Epoch: 016, Train Acc: 0.7733, Test Acc: 0.7794, Loss: 0.6862
Epoch: 0

In [77]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    ASAPooling,
    GraphConv,
    JumpingKnowledge,
    global_mean_pool,
)


class ASAP(torch.nn.Module):
    def __init__(self):
        super(ASAP, self).__init__()
        self.num_layers = 5 
        self.hidden  = 32
        self.ratio = 0.8
        self.dropout = 0.2
        self.conv1 = GraphConv(1, self.hidden, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(self.hidden, self.hidden, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend([
            ASAPooling(self.hidden, self.ratio, dropout=self.dropout)
            for i in range((self.num_layers) // 2)
        ])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(self.num_layers * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, edge_weight, batch = data.x, data.edge_index,data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = conv(x=x, edge_index=edge_index, edge_weight=edge_weight)
            x = F.relu(x)
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, edge_weight, batch, _ = pool(
                    x=x, edge_index=edge_index, edge_weight=edge_weight,
                    batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)


In [78]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ASAP().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.7266
Epoch: 002, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.6781
Epoch: 003, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.6858
Epoch: 004, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.6769
Epoch: 005, Train Acc: 0.5400, Test Acc: 0.4118, Loss: 0.6467
Epoch: 006, Train Acc: 0.8733, Test Acc: 0.8529, Loss: 0.5894
Epoch: 007, Train Acc: 0.6333, Test Acc: 0.5882, Loss: 0.5651
Epoch: 008, Train Acc: 0.7000, Test Acc: 0.7794, Loss: 0.5424
Epoch: 009, Train Acc: 0.8867, Test Acc: 0.8382, Loss: 0.4168
Epoch: 010, Train Acc: 0.8667, Test Acc: 0.8676, Loss: 0.3616
Epoch: 011, Train Acc: 0.9000, Test Acc: 0.8971, Loss: 0.2932
Epoch: 012, Train Acc: 0.8733, Test Acc: 0.8529, Loss: 0.8914
Epoch: 013, Train Acc: 0.9067, Test Acc: 0.9265, Loss: 0.2253
Epoch: 014, Train Acc: 0.9000, Test Acc: 0.8971, Loss: 0.3634
Epoch: 015, Train Acc: 0.9133, Test Acc: 0.8971, Loss: 0.4907
Epoch: 016, Train Acc: 0.9133, Test Acc: 0.8971, Loss: 0.3429
Epoch: 0